In [1]:
from vae_faces import FaceVAE
import torch
import matplotlib.pyplot as plt
import numpy as np
import glob
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets

docker is not installed. DockerTask requires docker.


In [2]:
def plot_lines(x, ax):
    ax.plot(x[0:17,  0],    -x[0:17,  1]) # Jaw
    ax.plot(x[17:22, 0],    -x[17:22, 1]) # left eyebrow
    ax.plot(x[22:27, 0],    -x[22:27, 1]) # right eyebrow
    ax.plot(x[27:31, 0],    -x[27:31, 1]) # vertical nose
    ax.plot(x[31:36, 0],    -x[31:36, 1]) # horizontal nose
    ax.plot(x[list(range(36,42)) + [36], 0], -x[list(range(36,42)) + [36], 1]) # left eye
    ax.plot(x[list(range(42, 48)) + [42], 0], -x[list(range(42, 48)) + [42], 1]) # right eye
    ax.plot(x[list(range(48, 60)) + [48], 0], -x[list(range(48, 60)) + [48], 1]) # outer mouth
    ax.plot(x[list(range(60, 68)) + [60], 0], -x[list(range(60, 68)) + [60], 1]) # inner mouth
    ax.scatter(x[68:70, 0], -x[68:70, 1])
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])

In [3]:
bottleneck_size = 5
experiment = 1

file_ = glob.glob(f'/workspace/results/model_weights/vae/{experiment}/_ckpt_epoch_*.ckpt')[0]

pretrained_model = FaceVAE.load_from_metrics(
    weights_path=file_,
    tags_csv=f'results/test_tube_data/vae/version_{experiment}/meta_tags.csv',
    on_gpu=False
#     map_location=None
)

# predict
pretrained_model.eval()
pretrained_model.freeze()


def f(**kwargs):
    sample = torch.tensor([x for x in kwargs.values()])

    x_out = pretrained_model.decode(sample)  # .numpy()
#     x_out = pretrained_model.reparameterize(mu, logvar)
    sample_res = x_out.reshape(70, 2).cpu()
    
    
    fig, ax1 = plt.subplots(1, 1)
    plot_lines(sample_res, ax1)
#     ax1.scatter(-sample_res[:, 0], -sample_res[:, 1])
#     plt.show()

varbs = {str(x): FloatSlider(min=-5, max=5, continuous_update=False) for x in range(bottleneck_size)}
interact(f, **varbs);

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='0', max=5.0, min=-5.0), Flo…